In [20]:
from collections import deque, defaultdict
import re

In [21]:
input_spec = open('input.txt').readlines()
test_spec = '''Begin in state A.
Perform a diagnostic checksum after 6 steps.

In state A:
  If the current value is 0:
    - Write the value 1.
    - Move one slot to the right.
    - Continue with state B.
  If the current value is 1:
    - Write the value 0.
    - Move one slot to the left.
    - Continue with state B.

In state B:
  If the current value is 0:
    - Write the value 1.
    - Move one slot to the left.
    - Continue with state A.
  If the current value is 1:
    - Write the value 1.
    - Move one slot to the right.
    - Continue with state A.'''.splitlines()

In [121]:
class UTM(object):
    def __init__(self):
        self.tape = defaultdict(int)
        self.cursor = 0
        self.state = None
        self.iters = 0
        self.stop = None
        self.spec = defaultdict(dict)
        
    def step(self):
        s = self.state
        c = self.tape[self.cursor]
        self.tape[self.cursor] = self.spec[s][c]['write']
        self.cursor += self.spec[s][c]['move']
        self.state = self.spec[s][c]['new_state']
        self.iters += 1
        return self
    
    def diagnose(self):
        while self.iters < self.stop:
            self.step()
        return self
        
    def checksum(self):
        return sum(self.tape.values())
    
    
    def load(self, speclines):
        self.state = re.match(r'Begin in state ([A-Z]).', speclines[0]).group(1)
        self.stop = int(re.match(r'Perform a diagnostic checksum after ([0-9]+) steps.', speclines[1]).group(1))
        
        state = current = new = slot_move = new_state = None
        for l in speclines[2:]:
            #print('l: ', l)
            m = re.match(r'In state ([A-Z]):', l)
            if m:
                state = m.group(1)
                #print('state', state)
                
            m = re.match(r'  If the current value is ([01]):', l)
            if m:
                current = int(m.group(1))
                #print('current', current)
                
            m = re.match(r'    - Write the value ([01]).', l)
            if m:
                new = int(m.group(1))
                #print('new', new)
                
            m = re.match(r'    - Move one slot to the (left|right).', l)
            if m:
                slot_move = -1 if m.group(1) == 'left' else 1
                
            m = re.match(r'    - Continue with state ([A-Z]).', l)
            if m:
                new_state = m.group(1)
                
            if new_state is not None:
                self.spec[state][current] = {'write': new, 'move': slot_move, 'new_state': new_state}
                current = new = slot_move = new_state = None
                
        return self
    

In [126]:
%%timeit -n1 -test = UTM()
test.load(test_spec)

print(f'test (should be 3): {test.diagnose().checksum()}')

test (should be 3): 3


In [125]:
part1 = UTM()
part1.load(input_spec)
print(f'part 1 answer: {part1.diagnose().checksum()}')

part 1 answer: 2870


In [128]:
min(part1.tape.keys())

-5581

In [129]:
max(part1.tape.keys())

18

In [130]:
18 + 5581

5599